
<br>
=========================================================<br>
Full tutorial on calibrating Denoisers Using J-Invariance<br>
=========================================================<br>
In this example, we show how to find an optimally calibrated<br>
version of any denoising algorithm.<br>
The calibration method is based on the `noise2self` algorithm of [1]_.<br>
.. [1] J. Batson & L. Royer. Noise2Self: Blind Denoising by Self-Supervision,<br>
       International Conference on Machine Learning, p. 524-533 (2019).<br>
.. seealso::<br>
   A simple example of the method is given in<br>
   :ref:`sphx_glr_auto_examples_filters_plot_j_invariant.py`.<br>


###################################################################<br>
Calibrating a wavelet denoiser

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec

In [ ]:
from skimage.data import chelsea, hubble_deep_field
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.restoration import (calibrate_denoiser,
                                 denoise_wavelet,
                                 denoise_tv_chambolle, denoise_nl_means,
                                 estimate_sigma)
from skimage.util import img_as_float, random_noise
from skimage.color import rgb2gray
from functools import partial

In [ ]:
_denoise_wavelet = partial(denoise_wavelet, rescale_sigma=True)

In [ ]:
image = img_as_float(chelsea())
sigma = 0.2
noisy = random_noise(image, var=sigma ** 2)

Parameters to test when calibrating the denoising algorithm

In [ ]:
parameter_ranges = {'sigma': np.arange(0.1, 0.3, 0.02),
                    'wavelet': ['db1', 'db2'],
                    'convert2ycbcr': [True, False],
                    'channel_axis': [-1]}

Denoised image using default parameters of `denoise_wavelet`

In [ ]:
default_output = denoise_wavelet(noisy, channel_axis=-1, rescale_sigma=True)

Calibrate denoiser

In [ ]:
calibrated_denoiser = calibrate_denoiser(noisy,
                                         _denoise_wavelet,
                                         denoise_parameters=parameter_ranges
                                         )

Denoised image using calibrated denoiser

In [ ]:
calibrated_output = calibrated_denoiser(noisy)

In [ ]:
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(15, 5))

In [ ]:
for ax, img, title in zip(axes,
                          [noisy, default_output, calibrated_output],
                          ['Noisy Image', 'Denoised (Default)',
                           'Denoised (Calibrated)']):
    ax.imshow(img)
    ax.set_title(title)
    ax.set_yticks([])
    ax.set_xticks([])

###################################################################<br>
The Self-Supervised Loss and J-Invariance<br>
=========================================<br>
The key to this calibration method is the notion of J-invariance. A denoising<br>
function is J-invariant if the prediction it makes for each pixel does<br>
not depend on the value of that pixel in the original image. The prediction<br>
for each pixel may instead use all the relevant information contained in the<br>
rest of the image, which is typically quite significant. Any function<br>
can be converted into a J-invariant one using a simple masking procedure,<br>
as described in [1].<br>
<br>
The pixel-wise error of a J-invariant denoiser is uncorrelated<br>
to the noise, so long as the noise in each pixel is independent.<br>
Consequently, the average difference between the denoised image and the<br>
noisy image, the *self-supervised loss*, is the same as the<br>
difference between the denoised image and the original clean image, the<br>
*ground-truth loss* (up to a constant).<br>
<br>
This means that the best J-invariant denoiser for a given image can<br>
be found using the noisy data alone, by selecting the denoiser minimizing<br>
the self-supervised loss. Below, we demonstrate this<br>
for a family of wavelet denoisers with varying `sigma` parameter. The<br>
self-supervised loss (solid blue line) and the ground-truth loss (dashed<br>
blue line) have the same shape and the same minimizer.<br>


In [ ]:
from skimage.restoration.j_invariant import _invariant_denoise

In [ ]:
sigma_range = np.arange(sigma/2, 1.5*sigma, 0.025)

In [ ]:
parameters_tested = [{'sigma': sigma, 'convert2ycbcr': True, 'wavelet': 'db2',
                      'channel_axis': -1}
                     for sigma in sigma_range]

In [ ]:
denoised_invariant = [_invariant_denoise(noisy, _denoise_wavelet,
                                         denoiser_kwargs=params)
                      for params in parameters_tested]

In [ ]:
self_supervised_loss = [mse(img, noisy) for img in denoised_invariant]
ground_truth_loss = [mse(img, image) for img in denoised_invariant]

In [ ]:
opt_idx = np.argmin(self_supervised_loss)
plot_idx = [0, opt_idx, len(sigma_range) - 1]

In [ ]:
get_inset = lambda x: x[25:225, 100:300]

In [ ]:
plt.figure(figsize=(10, 12))

In [ ]:
gs = gridspec.GridSpec(3, 3)
ax1 = plt.subplot(gs[0, :])
ax2 = plt.subplot(gs[1, :])
ax_image = [plt.subplot(gs[2, i]) for i in range(3)]

In [ ]:
ax1.plot(sigma_range, self_supervised_loss, color='C0',
         label='Self-Supervised Loss')
ax1.scatter(sigma_range[opt_idx], self_supervised_loss[opt_idx] + 0.0003,
            marker='v', color='red', label='optimal sigma')

In [ ]:
ax1.set_ylabel('MSE')
ax1.set_xticks([])
ax1.legend()
ax1.set_title('Self-Supervised Loss')

In [ ]:
ax2.plot(sigma_range, ground_truth_loss, color='C0', linestyle='--',
         label='Ground Truth Loss')
ax2.scatter(sigma_range[opt_idx], ground_truth_loss[opt_idx] + 0.0003,
            marker='v', color='red', label='optimal sigma')
ax2.set_ylabel('MSE')
ax2.legend()
ax2.set_xlabel('sigma')
ax2.set_title('Ground-Truth Loss')

In [ ]:
for i in range(3):
    ax = ax_image[i]
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(get_inset(denoised_invariant[plot_idx[i]]))
    ax.set_xlabel('sigma = ' + str(np.round(sigma_range[plot_idx[i]], 2)))

In [ ]:
for spine in ax_image[1].spines.values():
    spine.set_edgecolor('red')
    spine.set_linewidth(5)

###################################################################<br>
Conversion to J-invariance<br>
=========================================<br>
The function `_invariant_denoise` acts as a J-invariant version of a<br>
given denoiser. It works by masking a fraction of the pixels, interpolating<br>
them, running the original denoiser, and extracting the values returned in<br>
the masked pixels. Iterating over the image results in a fully J-invariant<br>
output.<br>
<br>
For any given set of parameters, the J-invariant version of a denoiser<br>
is different from the original denoiser, but it is not necessarily better<br>
or worse. In the plot below, we see that, for the test image of a cat,<br>
the J-invariant version of a wavelet denoiser is significantly better<br>
than the original at small values of variance-reduction `sigma` and<br>
imperceptibly worse at larger values.<br>


In [ ]:
parameters_tested = [{'sigma': sigma, 'convert2ycbcr': True,
                      'wavelet': 'db2', 'channel_axis': -1}
                     for sigma in sigma_range]

In [ ]:
denoised_original = [_denoise_wavelet(noisy, **params)
                     for params in parameters_tested]

In [ ]:
ground_truth_loss_invariant = [mse(img, image) for img in denoised_invariant]
ground_truth_loss_original = [mse(img, image) for img in denoised_original]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

In [ ]:
ax.plot(sigma_range, ground_truth_loss_invariant, color='C0', linestyle='--',
        label='J-invariant')
ax.plot(sigma_range, ground_truth_loss_original, color='C1', linestyle='--',
        label='Original')
ax.scatter(sigma_range[opt_idx], ground_truth_loss[opt_idx] + 0.001,
           marker='v', color='red')
ax.legend()
ax.set_title(
    'J-Invariant Denoiser Has Comparable Or '
    'Better Performance At Same Parameters'
)
ax.set_ylabel('MSE')
ax.set_xlabel('sigma')

###################################################################<br>
Comparing Different Classes of Denoiser<br>
=========================================<br>
The self-supervised loss can be used to compare different classes of<br>
denoiser in addition to choosing parameters for a single class.<br>
This allows the user to, in an unbiased way, choose the best parameters<br>
for the best class of denoiser for a given image.<br>
<br>
Below, we show this for an image of the hubble deep field with significant<br>
speckle noise added. In this case, the J-invariant calibrated denoiser is<br>
better than the default denoiser in each of three families of denoisers --<br>
Non-local means, wavelet, and TV norm. Additionally, the self-supervised<br>
loss shows that the TV norm denoiser is the best for this noisy image.<br>


In [ ]:
image = rgb2gray(img_as_float(hubble_deep_field()[100:250, 50:300]))

In [ ]:
sigma = 0.4
noisy = random_noise(image, mode='speckle', var=sigma ** 2)

In [ ]:
parameter_ranges_tv = {'weight': np.arange(0.01, 0.3, 0.02)}
_, (parameters_tested_tv, losses_tv) = calibrate_denoiser(
                                    noisy,
                                    denoise_tv_chambolle,
                                    denoise_parameters=parameter_ranges_tv,
                                    extra_output=True)
print(f'Minimum self-supervised loss TV: {np.min(losses_tv):.4f}')

In [ ]:
best_parameters_tv = parameters_tested_tv[np.argmin(losses_tv)]
denoised_calibrated_tv = _invariant_denoise(noisy, denoise_tv_chambolle,
                                            denoiser_kwargs=best_parameters_tv)
denoised_default_tv = denoise_tv_chambolle(noisy, **best_parameters_tv)

In [ ]:
psnr_calibrated_tv = psnr(image, denoised_calibrated_tv)
psnr_default_tv = psnr(image, denoised_default_tv)

In [ ]:
parameter_ranges_wavelet = {'sigma': np.arange(0.01, 0.3, 0.03)}
_, (parameters_tested_wavelet, losses_wavelet) = calibrate_denoiser(
                                                noisy,
                                                _denoise_wavelet,
                                                parameter_ranges_wavelet,
                                                extra_output=True)
print(f'Minimum self-supervised loss wavelet: {np.min(losses_wavelet):.4f}')

In [ ]:
best_parameters_wavelet = parameters_tested_wavelet[np.argmin(losses_wavelet)]
denoised_calibrated_wavelet = _invariant_denoise(
        noisy, _denoise_wavelet,
        denoiser_kwargs=best_parameters_wavelet)
denoised_default_wavelet = _denoise_wavelet(noisy, **best_parameters_wavelet)

In [ ]:
psnr_calibrated_wavelet = psnr(image, denoised_calibrated_wavelet)
psnr_default_wavelet = psnr(image, denoised_default_wavelet)

In [ ]:
sigma_est = estimate_sigma(noisy)

In [ ]:
parameter_ranges_nl = {'sigma': np.arange(0.6, 1.4, 0.2) * sigma_est,
                       'h': np.arange(0.6, 1.2, 0.2) * sigma_est}

In [ ]:
parameter_ranges_nl = {'sigma': np.arange(0.01, 0.3, 0.03)}
_, (parameters_tested_nl, losses_nl) = calibrate_denoiser(noisy,
                                                        denoise_nl_means,
                                                        parameter_ranges_nl,
                                                        extra_output=True)
print(f'Minimum self-supervised loss NL means: {np.min(losses_nl):.4f}')

In [ ]:
best_parameters_nl = parameters_tested_nl[np.argmin(losses_nl)]
denoised_calibrated_nl = _invariant_denoise(noisy, denoise_nl_means,
                                            denoiser_kwargs=best_parameters_nl)
denoised_default_nl = denoise_nl_means(noisy, **best_parameters_nl)

In [ ]:
psnr_calibrated_nl = psnr(image, denoised_calibrated_nl)
psnr_default_nl = psnr(image, denoised_default_nl)

In [ ]:
print(f'                       PSNR')
print(f'NL means (Default)   : {psnr_default_nl:.1f}')
print(f'NL means (Calibrated): {psnr_calibrated_nl:.1f}')
print(f'Wavelet  (Default)   : {psnr_default_wavelet:.1f}')
print(f'Wavelet  (Calibrated): {psnr_calibrated_wavelet:.1f}')
print(f'TV norm  (Default)   : {psnr_default_tv:.1f}')
print(f'TV norm  (Calibrated): {psnr_calibrated_tv:.1f}')

In [ ]:
plt.subplots(figsize=(10, 12))
plt.imshow(noisy, cmap='Greys_r')
plt.xticks([])
plt.yticks([])
plt.title('Noisy Image')

In [ ]:
get_inset = lambda x: x[0:100, -140:]

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 8))

In [ ]:
for ax in axes.ravel():
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
axes[0, 0].imshow(get_inset(denoised_default_nl), cmap='Greys_r')
axes[0, 0].set_title('NL Means Default')
axes[1, 0].imshow(get_inset(denoised_calibrated_nl), cmap='Greys_r')
axes[1, 0].set_title('NL Means Calibrated')
axes[0, 1].imshow(get_inset(denoised_default_wavelet), cmap='Greys_r')
axes[0, 1].set_title('Wavelet Default')
axes[1, 1].imshow(get_inset(denoised_calibrated_wavelet), cmap='Greys_r')
axes[1, 1].set_title('Wavelet Calibrated')
axes[0, 2].imshow(get_inset(denoised_default_tv), cmap='Greys_r')
axes[0, 2].set_title('TV Norm Default')
axes[1, 2].imshow(get_inset(denoised_calibrated_tv), cmap='Greys_r')
axes[1, 2].set_title('TV Norm Calibrated')

In [ ]:
for spine in axes[1, 2].spines.values():
    spine.set_edgecolor('red')
    spine.set_linewidth(5)

In [ ]:
plt.show()